In [ ]:
# Análisis de videojuegos - Tiendas ICE

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
df = pd.read_csv('/datasets/games.csv')

print(df.info())

print(df.head())
# Estandarizar nombres de columnas

df.columns = df.columns.str.lower()
# Convertir año a entero
df['year_of_release'] = pd.to_numeric(df['year_of_release'], errors='coerce').astype('Int64')
# Convertir user_score a numérico 
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

df.isnull().sum()
# La conversión de 'year_of_release' a entero ya que representa un año y no tiene caso mantenerlo como float,
#'user_score' se convirtió a númerico, reemplazando los valores "tbd" por NaN para que sean datos aún disponibles
# Los valores con críticas "tbd" se dejaron en blanco ya que no es relevante para el análisis
# Se podría quitar de la lista videojuegos que no tuvieran esta información, es posible que para análisis futuros se requieran de esos valores.

# Calcular ventas totales
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

games_per_year = df['year_of_release'].value_counts().sort_index()
games_per_year.plot(kind = 'bar', figsize = (12,6), title='Juegos lanzados por año')
plt.xlabel('Año de lanzamiento')
plt.ylabel('Cantidad de Juegos')
plt.show()

# Ventas por plataforma a lo largo de los años
top_platforms = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False).head(10)
top_platforms

# Evolución de las plataformas principales
popular_platforms = top_platforms.index
df_filtered = df[df['platform'].isin(popular_platforms)]
pivot = df_filtered.pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum')
pivot.plot(figsize=(12,6), title='Ventas por plataforma Top 10')
plt.ylabel('Ventas en millones de copias')
plt.show()

# Diagrama de caja de ventas por plataforma
sns.boxplot(data=df[df['platform'].isin(popular_platforms)], x='platform', y='total_sales')
plt.title('Distribución de ventas por plataforma')
plt.xticks(rotation=45)
plt.show()

# Reseñas vs ventas
wii = df[df['platform'] == 'Wii']
wii.plot(kind='scatter', x='critic_score', y='total_sales', title='Wii: Críticas vs Ventas')
print("Correlación crítica:", wii['critic_score'].corr(wii['total_sales']))
print("Correlación usuarios:", wii['user_score'].corr(wii['total_sales']))

# Reseñas vs ventas
xb360 = df[df['platform'] == 'X360']
xb360.plot(kind='scatter', x='critic_score', y='total_sales', title='Xbox 360: Críticas vs Ventas')
print("Correlación crítica:", xb360['critic_score'].corr(xb360['total_sales']))
print("Correlación usuarios:", xb360['user_score'].corr(xb360['total_sales']))

# Ventas por género
genre_sales = df.groupby('genre')['total_sales'].sum().sort_values(ascending=False)
genre_sales.plot(kind='bar', title='Ventas por género', figsize=(10,5))
plt.ylabel('Ventas globales')
plt.show()

# Principales plataformas por región
for region in ['na_sales', 'eu_sales', 'jp_sales']:
    print(f'\nTop plataformas en {region.upper()}:')
    print(df.groupby('platform')[region].sum().sort_values(ascending=False).head(5))

# Géneros proncipales por región
for region in ['na_sales', 'eu_sales', 'jp_sales']:
    print(f'\Top géneros en {region.upper()}:')
    print(df.groupby('genre')[region].sum().sort_values(ascending=False).head(5))

# Efecto de ESRB por región
for region in ['na_sales', 'eu_sales', 'jp_sales']:
    print(f'\Top géneros por ESRB en {region.upper()}:')
    print(df.groupby('rating')[region].sum().sort_values(ascending=False))

# Hipótesis 1: user_score en xbox one vs PC
xone_scores = df[df['platform'] == 'XOne']['user_score'].dropna()
pc_scores = df[df['platform'] == 'PC']['user_score'].dropna()

alpha = 0.05
results = ttest_ind(xone_scores, pc_scores)

print('p-valor', results.pvalue)
if results.pvalue < alpha:
    print("Rechazamos la hipótesis nula: hay diferencia")
else:
    print("No se puede rechazar la hipótesis nula: no hay diferencia significativa")

# Hipótesis 2: user_score en Acción vs Deportes

action = df[df['genre'] == 'Action']['user_score'].dropna()
sports = df[df['genre'] == 'Sports']['user_score'].dropna()

results2 = ttest_ind(action, sports)

print('p-valor:', results2.pvalue)
if results2.pvalue < alpha:
    print("Rechazamos la hipótesis nula no hay diferencia significativa")
else: 
    print("No se puede rechazar la hipótesis nula: no hay diferencia significativa")

La formulación de hipótesis es la misma que el caso anterior, evidentemente con base en los generos de acción y deportes. Como dato adicional se utilizó un alpha estándar que se ha venido utilizando durante el aprendizaje.

Conclusiones: -Las plataformas con más ventas son PS2, PS3, PS4 y Xbox 360 -Los géneros más rentables son acción y deporte -En Japón domina RPG y DS, mientras que en EU y NA domina Acción -Las reseñas de críticos tienen mayor relación con las ventas -Si hay diferencia significativa entre las calificaciones de usuarios de Xbox Oney PC -No hay diferencia entre los géneros de Acción y Deporte en las calificaciones de usuarios
